# Bias Correction for Regional Climate Models

This notebook demonstrates statistical bias correction techniques for CORDEX regional climate projections.

## Why Bias Correction?

Climate models systematically deviate from observations due to:
- Imperfect representation of physical processes
- Spatial resolution limitations
- Parameterization uncertainties

Bias correction adjusts model outputs to match observational statistics while preserving the climate change signal.

## Methods Covered

1. **Quantile Mapping (QM)**: Maps model distribution to observations
2. **Delta Method**: Applies model change signal to observations
3. **Seasonal Corrections**: Applied separately by season

## Learning Objectives

- Understand and implement bias correction algorithms
- Validate correction effectiveness
- Preserve climate change signals
- Apply corrections to temperature and precipitation

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from scipy import interpolate

warnings.filterwarnings("ignore")

plt.style.use("seaborn-v0_8-darkgrid")
%matplotlib inline

print("✓ Imports complete")

## Create Synthetic Data with Realistic Bias

For demonstration, we'll create:
- **Observations**: Reference climate data
- **Model Historical**: Biased version of observations
- **Model Future**: Historical + climate change signal

In [ ]:
# Generate synthetic data
np.random.seed(42)
n_years = 30
n_days = n_years * 365

# Time arrays
time_hist = pd.date_range("1975-01-01", periods=n_days, freq="D")
time_fut = pd.date_range("2070-01-01", periods=n_days, freq="D")

# Day of year for seasonality
doy = np.tile(np.arange(1, 366), n_years)

# OBSERVATIONS (truth)
# Temperature: 15°C mean, 10°C seasonal amplitude
obs_temp = 15 + 10 * np.sin(2 * np.pi * doy / 365) + np.random.normal(0, 2, n_days)

# MODEL HISTORICAL (with bias)
# Warm bias: +2°C mean, reduced seasonal amplitude
model_hist_temp = 17 + 8 * np.sin(2 * np.pi * doy / 365) + np.random.normal(0, 2.5, n_days)

# MODEL FUTURE (historical + warming signal)
# Add 3°C warming trend
warming_signal = 3.0
model_fut_temp = model_hist_temp + warming_signal + np.random.normal(0, 0.5, n_days)

# Create xarray datasets
ds_obs = xr.Dataset({"tas": (["time"], obs_temp)}, coords={"time": time_hist})

ds_model_hist = xr.Dataset({"tas": (["time"], model_hist_temp)}, coords={"time": time_hist})

ds_model_fut = xr.Dataset({"tas": (["time"], model_fut_temp)}, coords={"time": time_fut})

print(f"Generated {n_years} years of daily data")
print(f"Observations mean: {obs_temp.mean():.2f}°C")
print(f"Model historical mean: {model_hist_temp.mean():.2f}°C")
print(f"Model future mean: {model_fut_temp.mean():.2f}°C")

## Visualize Raw Model Bias

Compare model historical run with observations to identify systematic biases.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Monthly means
obs_monthly = ds_obs.groupby("time.month").mean()
model_monthly = ds_model_hist.groupby("time.month").mean()

axes[0, 0].plot(range(1, 13), obs_monthly["tas"], "o-", label="Observations", linewidth=2)
axes[0, 0].plot(range(1, 13), model_monthly["tas"], "s-", label="Model", linewidth=2)
axes[0, 0].set_xlabel("Month")
axes[0, 0].set_ylabel("Temperature (°C)")
axes[0, 0].set_title("Monthly Mean Temperature")
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Bias by month
bias_monthly = model_monthly["tas"] - obs_monthly["tas"]
axes[0, 1].bar(range(1, 13), bias_monthly, color="coral", edgecolor="black")
axes[0, 1].axhline(0, color="black", linestyle="--", linewidth=1)
axes[0, 1].set_xlabel("Month")
axes[0, 1].set_ylabel("Bias (°C)")
axes[0, 1].set_title("Model Bias by Month")
axes[0, 1].grid(True, alpha=0.3, axis="y")

# Distribution comparison
axes[1, 0].hist(ds_obs["tas"], bins=50, alpha=0.5, label="Observations", density=True)
axes[1, 0].hist(ds_model_hist["tas"], bins=50, alpha=0.5, label="Model", density=True)
axes[1, 0].set_xlabel("Temperature (°C)")
axes[1, 0].set_ylabel("Density")
axes[1, 0].set_title("Temperature Distribution")
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis="y")

# Q-Q plot
obs_sorted = np.sort(ds_obs["tas"].values)
model_sorted = np.sort(ds_model_hist["tas"].values)
axes[1, 1].scatter(obs_sorted[::100], model_sorted[::100], alpha=0.5, s=10)
axes[1, 1].plot(
    [obs_sorted.min(), obs_sorted.max()],
    [obs_sorted.min(), obs_sorted.max()],
    "r--",
    label="1:1 line",
)
axes[1, 1].set_xlabel("Observed Temperature (°C)")
axes[1, 1].set_ylabel("Model Temperature (°C)")
axes[1, 1].set_title("Q-Q Plot")
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean bias: {(ds_model_hist['tas'].mean() - ds_obs['tas'].mean()).values:.2f}°C")
print(f"Std bias: {(ds_model_hist['tas'].std() - ds_obs['tas'].std()).values:.2f}°C")

## Quantile Mapping Bias Correction

### Method

Quantile mapping matches the cumulative distribution function (CDF) of model data to observations:

$$T_{corrected} = F_{obs}^{-1}(F_{model}(T_{model}))$$

Where:
- $F_{model}$: Model CDF (from historical period)
- $F_{obs}^{-1}$: Inverse observation CDF
- Applied to both historical and future periods

In [ ]:
def quantile_mapping(model_hist, obs, model_fut, n_quantiles=1000):
    """
    Apply empirical quantile mapping bias correction.

    Parameters:
    -----------
    model_hist : array
        Model historical period
    obs : array
        Observations (same period as model_hist)
    model_fut : array
        Model future period to be corrected
    n_quantiles : int
        Number of quantiles for mapping

    Returns:
    --------
    corrected : array
        Bias-corrected future values
    """
    # Calculate quantiles
    quantiles = np.linspace(0, 1, n_quantiles)

    # Build transfer function from historical period
    model_hist_quantiles = np.quantile(model_hist, quantiles)
    obs_quantiles = np.quantile(obs, quantiles)

    # Create interpolation function
    # This maps model values to observation values
    transfer_func = interpolate.interp1d(
        model_hist_quantiles,
        obs_quantiles,
        bounds_error=False,
        fill_value=(obs_quantiles[0], obs_quantiles[-1]),
    )

    # Apply to future period
    corrected = transfer_func(model_fut)

    return corrected


# Apply quantile mapping
tas_corrected_qm = quantile_mapping(
    ds_model_hist["tas"].values, ds_obs["tas"].values, ds_model_fut["tas"].values
)

# Create corrected dataset
ds_corrected_qm = xr.Dataset({"tas": (["time"], tas_corrected_qm)}, coords={"time": time_fut})

print("✓ Quantile mapping complete")
print(f"Original future mean: {ds_model_fut['tas'].mean().values:.2f}°C")
print(f"Corrected future mean: {ds_corrected_qm['tas'].mean().values:.2f}°C")
print(f"Expected (obs + warming): {ds_obs['tas'].mean().values + warming_signal:.2f}°C")

## Delta Method Bias Correction

### Method

The delta (change factor) method applies the model's climate change signal to observations:

For temperature (additive):
$$T_{corrected} = T_{obs,hist} + (T_{model,fut} - T_{model,hist})$$

For precipitation (multiplicative):
$$P_{corrected} = P_{obs,hist} \times \frac{P_{model,fut}}{P_{model,hist}}$$

In [ ]:
def delta_method(model_hist, obs, model_fut, method="additive"):
    """
    Apply delta method bias correction.

    Parameters:
    -----------
    model_hist : array
        Model historical period
    obs : array
        Observations
    model_fut : array
        Model future period
    method : str
        'additive' for temperature, 'multiplicative' for precipitation

    Returns:
    --------
    corrected : array
        Bias-corrected future values
    """
    if method == "additive":
        # Calculate change signal
        delta = model_fut - model_hist.mean()
        # Apply to observations
        corrected = obs.mean() + delta
    elif method == "multiplicative":
        # Calculate change factor
        factor = model_fut / model_hist.mean()
        # Apply to observations
        corrected = obs.mean() * factor

    return corrected


# Apply delta method
tas_corrected_delta = delta_method(
    ds_model_hist["tas"].values, ds_obs["tas"].values, ds_model_fut["tas"].values, method="additive"
)

ds_corrected_delta = xr.Dataset({"tas": (["time"], tas_corrected_delta)}, coords={"time": time_fut})

print("✓ Delta method complete")
print(f"Corrected future mean: {ds_corrected_delta['tas'].mean().values:.2f}°C")

## Compare Correction Methods

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Time series comparison
sample_days = slice(0, 365 * 3)  # First 3 years
axes[0, 0].plot(
    ds_model_fut["time"][sample_days],
    ds_model_fut["tas"][sample_days],
    alpha=0.3,
    label="Raw Model",
    linewidth=1,
)
axes[0, 0].plot(
    ds_corrected_qm["time"][sample_days],
    ds_corrected_qm["tas"][sample_days],
    alpha=0.7,
    label="QM Corrected",
    linewidth=1,
)
axes[0, 0].plot(
    ds_corrected_delta["time"][sample_days],
    ds_corrected_delta["tas"][sample_days],
    alpha=0.7,
    label="Delta Corrected",
    linewidth=1,
)
axes[0, 0].set_xlabel("Time")
axes[0, 0].set_ylabel("Temperature (°C)")
axes[0, 0].set_title("Time Series: First 3 Years")
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Distribution comparison
axes[0, 1].hist(ds_model_fut["tas"], bins=50, alpha=0.3, label="Raw Model", density=True)
axes[0, 1].hist(ds_corrected_qm["tas"], bins=50, alpha=0.5, label="QM Corrected", density=True)
axes[0, 1].hist(
    ds_corrected_delta["tas"], bins=50, alpha=0.5, label="Delta Corrected", density=True
)
axes[0, 1].set_xlabel("Temperature (°C)")
axes[0, 1].set_ylabel("Density")
axes[0, 1].set_title("Temperature Distribution")
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3, axis="y")

# Monthly climatology
model_monthly_fut = ds_model_fut.groupby("time.month").mean()
qm_monthly = ds_corrected_qm.groupby("time.month").mean()
delta_monthly = ds_corrected_delta.groupby("time.month").mean()

axes[1, 0].plot(range(1, 13), model_monthly_fut["tas"], "o-", label="Raw Model", linewidth=2)
axes[1, 0].plot(range(1, 13), qm_monthly["tas"], "s-", label="QM Corrected", linewidth=2)
axes[1, 0].plot(range(1, 13), delta_monthly["tas"], "^-", label="Delta Corrected", linewidth=2)
axes[1, 0].set_xlabel("Month")
axes[1, 0].set_ylabel("Temperature (°C)")
axes[1, 0].set_title("Monthly Mean Temperature (Future)")
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Summary statistics
stats_data = [
    [
        "Raw Model",
        ds_model_fut["tas"].mean().values,
        ds_model_fut["tas"].std().values,
        np.percentile(ds_model_fut["tas"], 5),
        np.percentile(ds_model_fut["tas"], 95),
    ],
    [
        "QM Corrected",
        ds_corrected_qm["tas"].mean().values,
        ds_corrected_qm["tas"].std().values,
        np.percentile(ds_corrected_qm["tas"], 5),
        np.percentile(ds_corrected_qm["tas"], 95),
    ],
    [
        "Delta Corrected",
        ds_corrected_delta["tas"].mean().values,
        ds_corrected_delta["tas"].std().values,
        np.percentile(ds_corrected_delta["tas"], 5),
        np.percentile(ds_corrected_delta["tas"], 95),
    ],
]

stats_df = pd.DataFrame(stats_data, columns=["Method", "Mean", "Std", "P5", "P95"])
axes[1, 1].axis("off")
table = axes[1, 1].table(
    cellText=stats_df.values, colLabels=stats_df.columns, cellLoc="center", loc="center"
)
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)
axes[1, 1].set_title("Summary Statistics (°C)", pad=20)

plt.tight_layout()
plt.show()

## Validation: Climate Change Signal Preservation

Critical check: Bias correction should not alter the climate change signal.

In [ ]:
# Calculate climate change signals
raw_signal = ds_model_fut["tas"].mean() - ds_model_hist["tas"].mean()
qm_signal = ds_corrected_qm["tas"].mean() - ds_obs["tas"].mean()
delta_signal = ds_corrected_delta["tas"].mean() - ds_obs["tas"].mean()

print("Climate Change Signals:")
print(f"Raw model signal: {raw_signal.values:.2f}°C")
print(f"QM corrected signal: {qm_signal.values:.2f}°C")
print(f"Delta corrected signal: {delta_signal.values:.2f}°C")
print(f"\nTrue warming: {warming_signal:.2f}°C")
print(f"\n✓ QM preserves signal: {abs(qm_signal.values - warming_signal) < 0.5}")
print(f"✓ Delta preserves signal: {abs(delta_signal.values - warming_signal) < 0.5}")

## Save Corrected Data

In [ ]:
# Add metadata
ds_corrected_qm["tas"].attrs = {
    "long_name": "Near-surface air temperature",
    "units": "degC",
    "bias_correction": "Quantile Mapping",
    "description": "Bias-corrected using empirical quantile mapping",
}

# Save to NetCDF
output_path = "../data/processed/tas_bias_corrected_qm.nc"
ds_corrected_qm.to_netcdf(output_path, encoding={"tas": {"zlib": True, "complevel": 5}})

print(f"✓ Saved bias-corrected data to {output_path}")
print("\nReady for further analysis!")

## Summary

### Key Findings

1. **Bias Identification**: Model shows systematic warm bias of ~2°C
2. **Correction Effectiveness**: Both QM and Delta methods successfully remove bias
3. **Signal Preservation**: Climate change signal maintained after correction
4. **Method Comparison**:
   - **Quantile Mapping**: Best for correcting full distribution
   - **Delta Method**: Simpler, good for mean changes

### Best Practices

- Apply corrections separately by season for better results
- Use QM for temperature and precipitation extremes
- Use Delta method for simple trend analysis
- Always validate that climate change signal is preserved

### Next Steps

1. **Notebook 03**: Trend analysis of corrected projections
2. **Notebook 04**: Extreme event analysis
3. **Notebook 05**: Sector impact assessment